In [107]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import time
import re
import os
import sqlite3

import requests
import yfinance as yf

import settings


In [109]:
def print_info(current_index, extra_info, start_time, total_size):
    """
    Prints the provided information along with the progress, elapsed time, and remaining time.

    Parameters:
    - current_index (int): The current index of the item being processed.
    - extra_info (list): The extra information extracted containing multiple values.
    - start_time (float): The start time of the process.
    - total_size (int): The total number of items to be processed.
    """
    completed_items = current_index + 1
    remaining_items = total_size - completed_items
    percentage_completed = completed_items / total_size
    
    elapsed_time = time.time() - start_time
    avg_time_per_item = elapsed_time / completed_items
    remaining_time = remaining_items * avg_time_per_item
    
    elapsed_hours, elapsed_remainder = divmod(int(elapsed_time), 3600)
    elapsed_minutes, elapsed_seconds = divmod(elapsed_remainder, 60)
    elapsed_time_formatted = f'{int(elapsed_hours)}h {int(elapsed_minutes):02}m {int(elapsed_seconds):02}s'

    remaining_hours, remaining_remainder = divmod(int(remaining_time), 3600)
    remaining_minutes, remaining_seconds = divmod(remaining_remainder, 60)
    remaining_time_formatted = f'{int(remaining_hours)}h {int(remaining_minutes):02}m {int(remaining_seconds):02}s'
    
    progress = (
        f'{percentage_completed:.2%} '
        f'{completed_items}+{remaining_items}, '
        f'{avg_time_per_item:.6f}s per item, '
        f'Time: {elapsed_time_formatted} + {remaining_time_formatted}'
    )
    
    extra_info_str = " ".join(map(str, extra_info))
    print(f"{progress} {extra_info_str}")
    # winbeep()


### Math

In [ ]:
df = pd.read_csv('../df.csv')
df['quarter'] = pd.to_datetime(df['quarter'])
df['year'] = df['quarter'].dt.year
df['month'] = df['quarter'].dt.month
df['value'] = df['value']/1000000000



In [ ]:
# Create a pivot table for the 'value' column
value_pivot = df.pivot_table(
    index=['company_name', 'type', 'frame', 'account', 'year'],
    columns='month',
    values='value',
    aggfunc='first'
).reset_index()

# Renaming the columns to match the desired format
value_pivot.columns.name = None
value_pivot.columns = ['company_name', 'type', 'frame', 'account', 'year', 3, 6, 9, 12]

# Merge the pivoted data back with the original dataframe on key columns
final_df = pd.merge(
    df.drop(columns=['nsd', 'version', 'value', 'quarter', 'month']),  # Drop 'value', 'quarter', 'month' columns to avoid duplication
    value_pivot,
    left_on=['company_name', 'type', 'frame', 'account', 'year'],
    right_on=['company_name', 'type', 'frame', 'account', 'year'],
    how='left'
)

# # Removing duplicates based on all columns to keep the first occurrence
final_df = final_df.drop_duplicates(subset=['company_name', 'type', 'frame', 'account', 'year'])

# mathmagic
final_df[12] = final_df[12] - (final_df[9] + final_df[6] + final_df[3])



In [ ]:
# Create a new DataFrame by unpivoting the columns 3, 6, 9, and 12
unpivoted_df = final_df.melt(
    id_vars=['company_name', 'type', 'frame', 'account', 'year'],  # Columns to keep
    value_vars=[3, 6, 9, 12],  # Columns to unpivot
    var_name='month',  # Name for the 'variable' column
    value_name='value'  # Name for the 'value' column
)

# Create the 'quarter' column based on 'year' and 'month'
unpivoted_df['quarter'] = pd.to_datetime(unpivoted_df['year'].astype(str) + '-' + unpivoted_df['month'].astype(str) + '-01') + pd.offsets.MonthEnd(0)

# Sort the filtered DataFrame by company_name and quarter for better readability
unpivoted_df = unpivoted_df.sort_values(by=['company_name', 'quarter']).reset_index(drop=True)

# Filter the unpivoted DataFrame based on the specified conditions
filtered_unpivoted_df = unpivoted_df.loc[
    (unpivoted_df['company_name'] == 'TELEFONICA BRASIL SA') &
    (unpivoted_df['type'] == 'DFs Consolidadas') &
    (unpivoted_df['account'] == '3.01')
]

filtered_unpivoted_df


In [ ]:
# Merge filtered_unpivoted_df with the original df on common columns to incorporate additional columns
merged_df = pd.merge(
    unpivoted_df,
    df[['nsd', 'sector', 'subsector', 'segment', 'company_name', 'quarter', 'version', 'type', 'frame', 'account', 'description']],
    on=['company_name', 'type', 'frame', 'account', 'quarter'],
    how='right'
)[df.columns]



In [ ]:
# Apply filter conditions for 'company_name', 'type', and 'account'
merged_df = merged_df.loc[
    (merged_df['type'] == 'DFs Consolidadas') &
    (merged_df['account'] == '3.01')
]

# Create a Plotly line plot by company
fig = px.line(
    merged_df,
    x='quarter',  # x-axis is the 'quarter'
    y='value',    # y-axis is the 'value'
    color='company_name',  # Different lines for each company
    title='Value Over Time by Quarter for Each Company',  # Title of the plot
    labels={'quarter': 'Quarter', 'value': 'Value', 'company_name': 'Company'},  # Axis labels and legend title
)

# Show the plot
fig.show()


In [ ]:
df = pd.read_csv('../COMUNICACOES_math.csv')
df['quarter'] = pd.to_datetime(df['quarter'])


In [ ]:
# Pivot the DataFrame by 'quarter'
pivoted_df = df.pivot_table(
    index=['sector', 'subsector', 'segment', 'company_name', 'type', 'frame', 'account', 'description'],  # Keep these columns as index
    columns='quarter',  # Pivot on the 'quarter' column
    values='value',  # Use 'value' as the values to fill in
    aggfunc='sum'  # Aggregation function to handle multiple entries per group
)

# Flatten the multi-level column index after pivot
pivoted_df.columns = pivoted_df.columns.strftime('%Y-%m-%d')  # Format datetime columns for readability

# Reset index to convert the pivot table back into a DataFrame
pivoted_df.reset_index(inplace=True)


### Standardization

In [53]:
def apply_criteria(df, criteria, parent_mask=None, output_file='output.txt', parent_criteria_info=None):
    """
    Applies a criterion and its sub-criteria to the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to be modified.
        criteria (dict): A dictionary containing 'target', 'filter', and 'sub_criteria'.
        parent_mask (pd.Series): Optional parent level mask to apply.
        output_file (str): Path to the output file.
        parent_criteria_info (list): List to keep track of parent criteria for output purposes.

    Returns:
        pd.DataFrame: The modified DataFrame.
    """
    target = criteria['target']
    filters = criteria['filter']
    sub_criteria = criteria.get('sub_criteria', [])
    
    # Initialize the parent criteria info list if not provided
    if parent_criteria_info is None:
        parent_criteria_info = []

    # Initialize the mask for the entire DataFrame or use the parent mask
    base_mask = pd.Series([True] * len(df)) if parent_mask is None else parent_mask.copy()
    mask = base_mask.copy()

    # Append the current criteria to the parent criteria info
    parent_criteria_info.append({'target': target, 'filters': filters})

    crits = []

    # Define a dictionary to map filter conditions to functions
    condition_map = {
        'equals': lambda col, val: col == val.lower(),
        'not_equals': lambda col, val: col != val.lower(),
        # 'startswith': lambda col, val: col.str.startswith(tuple(map(str.lower, val))),
        'startswith': lambda col, val: col.astype(str).str.startswith(val),
        'not_startswith': lambda col, val: ~col.str.startswith(tuple(map(str.lower, val))),
        'endswith': lambda col, val: col.str.endswith(tuple(map(str.lower, val))),
        'not_endswith': lambda col, val: ~col.str.endswith(tuple(map(str.lower, val))),
        'contains_all': lambda col, val: col.apply(lambda x: all(term in x.lower() for term in val) if pd.notna(x) else False),
        'contains_any': lambda col, val: col.str.contains('|'.join(map(re.escape, val)), case=False, na=False),
        'contains_none': lambda col, val: ~col.str.contains('|'.join(map(re.escape, val)), case=False, na=False),
        'not_contains': lambda col, val: col.apply(lambda x: not all(term in x.lower() for term in val) if pd.notna(x) else True),
        'not_contains_any': lambda col, val: col.apply(lambda x: all(term not in x.lower() for term in val) if pd.notna(x) else True),
        'level': lambda col, val: (col.str.count(r'\.') + 1) == int(val)  # Merged level filter for exact levels
    }

    # Apply each filter to the mask
    for filter_column, filter_condition, filter_value in filters:
        crits.append([filter_column, filter_condition, filter_value])

        # Convert filter values to lists for conditions that need lists
        if filter_condition in ['contains_any', 'contains_none', 'contains_all', 'not_contains_all']:
            if not isinstance(filter_value, list):
                filter_value = [filter_value]

        # df_column_lower = df[filter_column].str.lower() if df[filter_column].dtype == 'O' else df[filter_column]
        df_column_lower = df[filter_column].str.lower().str.strip() if df[filter_column].dtype == 'O' else df[filter_column]

        # Apply the filter condition using the mapping
        if filter_condition in condition_map:
            condition_result = condition_map[filter_condition](df_column_lower, filter_value)
            mask &= condition_result
        else:
            raise ValueError(f"Unknown filter condition: {filter_condition}")

    # Ensure 'account_standard', 'description_standard', 'standard_criteria', and 'items_match' columns exist
    if 'account_standard' not in df.columns:
        df['account_standard'] = ''
    if 'description_standard' not in df.columns:
        df['description_standard'] = ''
    if 'standard_criteria' not in df.columns:
        df['standard_criteria'] = ''
    if 'items_match' not in df.columns:
        df['items_match'] = ''

    # Apply the target modifications to the DataFrame for the current mask
    account, description = target.split(' - ')
    df.loc[mask, 'account_standard'] = account
    df.loc[mask, 'description_standard'] = description
    df.loc[mask, 'standard_criteria'] = ' | '.join([f"{c[0]} {c[1]} {c[2]}" for c in crits])  # Add criteria details
    
    # Capture "contém itens como" (items that match the mask)
    items_example = df.loc[mask, ['account', 'description']].drop_duplicates().apply(lambda row: f"{row['account']} - {row['description']}", axis=1).tolist()
    df.loc[mask, 'items_match'] = ', '.join(items_example)

    # Write the results to the output file and print to console
    with open(output_file, 'a') as f:
        # 1. Print target (account - description)
        print(f"{account} - {description}")
        f.write(f"{account} - {description}\n")

        # 3. Print Criteria Path with target included
        print('Criteria Path:')
        f.write('Criteria Path:\n')
        for parent in parent_criteria_info:
            print(f"  {parent['target']}")
            f.write(f"  {parent['target']}\n")
            for filt in parent['filters']:
                print(f"    {filt}")
                f.write(f"    {filt}\n")
        
        # 2. Print "Contém itens como:"
        print('Contém itens como:')
        f.write('Contém itens como:\n')
        for item in items_example:
            print(f"    {item}")
            f.write(f"    {item}\n")
       
        # Blank line after Criteria Path
        print('')
        f.write('\n')

        # Two blank lines after each section
        print('\n')
        f.write('\n\n')

    # Recursively apply subcriteria using a new refined mask
    for sub in sub_criteria:
        # Create a new mask for sub-criteria by filtering the df with 'startswith' of the current filtered results
        sub_accounts = df.loc[mask, 'account'].unique()
        sub_mask = df['account'].apply(lambda x: any(x.startswith(acct) for acct in sub_accounts))
        
        apply_criteria(df, sub, parent_mask=sub_mask, output_file=output_file, parent_criteria_info=parent_criteria_info.copy())

    return df


def apply_criteria_tree(df, criteria_tree, output_file='output.txt'):
    """
    Main function to apply a criteria tree to the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to be modified.
        criteria_tree (list): List of criteria in a tree format.
        output_file (str): Path to the output file.

    Returns:
        pd.DataFrame: The modified DataFrame after applying all criteria.
    """
    for criteria in criteria_tree:
        df = apply_criteria(df, criteria, output_file=output_file)
    return df


In [58]:
section_0_criteria = [
    {
        'target': '00.01.01 - Ações ON Ordinárias',
        'filter': [
            ['account', 'equals', '00.01.01'],
            ['description', 'equals', 'Ações ON Ordinárias']
        ],
        'sub_criteria': []  # No sub-criteria for this example
    },
    {
        'target': '00.01.02 - Ações PN Preferenciais',
        'filter': [
            ['account', 'equals', '00.01.02'],
            ['description', 'equals', 'Ações PN Preferenciais']
        ],
        'sub_criteria': []  # No sub-criteria for this example
    },
    {
        'target': '00.02.01 - Em Tesouraria Ações ON Ordinárias',
        'filter': [
            ['account', 'equals', '00.02.01'],
            ['description', 'equals', 'Em Tesouraria Ações ON Ordinárias']
        ],
        'sub_criteria': []  # No sub-criteria for this example
    },
    {
        'target': '00.02.02 - Em Tesouraria Ações PN Preferenciais',
        'filter': [
            ['account', 'equals', '00.02.02'],
            ['description', 'equals', 'Em Tesouraria Ações PN Preferenciais']
        ],
        'sub_criteria': []  # No sub-criteria for this example
    }
]


In [57]:
section_1_criteria = [
    {
        'target': '01 - Ativo Total',
        'filter': [
            ['account', 'level', 1],
            ['account', 'startswith', '1']
        ],
        'sub_criteria': [
            {
                'target': '01.01 - Ativo Circulante de Curto Prazo',
                'filter': [
                    ['account', 'level', 2],
                    ['account', 'startswith', '1.01'],
                    ['description', 'contains_all', 'circul'],
                    ['description', 'not_contains', 'não']
                ],
                'sub_criteria': [
                    {
                        'target': '01.01.01 - Caixa e Equivalentes de Caixa de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'caixa']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.01.01.01 - Caixa e Bancos de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.01'],
                                    ['description', 'contains_all', 'caix']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.01.01.02 - Aplicações Líquidas de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.01'],
                                    ['description', 'contains_all', 'aplic']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.01.02 - Aplicações Financeiras de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'aplica']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.01.02.01 - Aplicações a Valor Justo de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.02'],
                                    ['description', 'contains_all', 'just']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.01.02.02 - Aplicações ao Custo Amortizado de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.02'],
                                    ['description', 'contains_all', 'cust']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.01.03 - Contas a Receber de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'receb']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.01.03.01 - Contas de Clientes de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.03'],
                                    ['description', 'contains_all', 'client']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '01.01.03.01.01 - Clientes',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.01.03.01'],
                                            ['description', 'contains_all', 'client']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.01.03.01.02 - Créditos de Liquidação Duvidosa',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.01.03.01'],
                                            ['description', 'contains_all', 'duvid']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.01.03.01.03 - Outros',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.01.03.01'],
                                            ['description', 'contains_none', ['client', 'duvid']]
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '01.01.03.02 - Outras Contas de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.03'],
                                    ['description', 'not_contains', 'client']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.01.04 - Estoques de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'estoq']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.01.04.01 - Estoques de Material de Consumo de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.04'],
                                    ['description', 'contains_all', 'mater']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.01.04.02 - Estoques de Material para Revenda de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.04'],
                                    ['description', 'contains_all', 'revend']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.01.04.03 - Estoques de Outros Itens de Curto Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.01.04'],
                                    ['description', 'contains_none', ['mater', 'revend']]
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.01.05 - Ativos Biológicos de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'biol']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '01.01.06 - Tributos a Recuperar de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'tribut']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '01.01.07 - Despesas Antecipadas de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'despes']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '01.01.09 - Outros Ativos Circulantes de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.01'],
                            ['description', 'contains_all', 'outros']
                        ],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '01.02 - Ativo Não Circulante de Longo Prazo',
                'filter': [
                    ['account', 'level', 2],
                    ['account', 'startswith', '1.02'],
                    ['description', 'contains_all', ['não', 'circul']]
                ],
                'sub_criteria': [
                    {
                        'target': '01.02.01 - Ativo Realizável a Longo Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.02'],
                            ['description', 'contains_all', 'longo prazo']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.02.01.02 - Aplicações a Valor Justo de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'aplic']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.03 - Contas a Receber de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'cont']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.04 - Estoques de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'estoq']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.05 - Ativos Biológicos de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'biol']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.06 - Tributos a Recuperar de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'tribut']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.07 - Despesas Antecipadas de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'despes']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.08 - Créditos com Partes Relacionadas de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'relacionad']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.01.09 - Outros Ativos Circulantes de Longo Prazo',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.01'],
                                    ['description', 'contains_all', 'outros'],
                                    ['description', 'not_contains', 'aplic']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.02.02 - Investimentos',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.02'],
                            ['description', 'contains_all', 'invest']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.02.02.01 - Participações Societárias',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.02'],
                                    ['description', 'contains_all', 'particip']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '01.02.02.01.01 - Participações em Coligadas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.02.01'],
                                            ['description', 'contains_all', 'colig']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.02.02.01.02 - Participações em Controladas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.02.01'],
                                            ['description', 'contains_all', 'control']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.02.02.01.03 - Outras',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.02.01'],
                                            ['description', 'contains_none', ['colig', 'control']]
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '01.02.02.02 - Propriedades para Investimento',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.02'],
                                    ['description', 'contains_all', 'propried']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.02.03 - Imobilizado',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.02'],
                            ['description', 'contains_all', 'imob']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.02.03.01 - Imobilizado em Operação',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.03'],
                                    ['description', 'contains_all', 'imobiliz']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '01.02.03.02 - Direito de Uso em Arrendamento',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.03'],
                                    ['description', 'contains_all', 'direito']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '01.02.04 - Intangível',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '1.02'],
                            ['description', 'contains_all', 'intang']
                        ],
                        'sub_criteria': [
                            {
                                'target': '01.02.04.01 - Intangíveis',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.04'],
                                    ['description', 'contains_all', 'intang']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '01.02.04.01.01 - Carteira de Clientes',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.04.01'],
                                            ['description', 'contains_all', 'client']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.02.04.01.02 - Softwares',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.04.01'],
                                            ['description', 'contains_any', ['softwar', 'aplicativ', 'sistem']]
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.02.04.01.03 - Marcas e Patentes',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.04.01'],
                                            ['description', 'contains_any', ['marc', 'patent']]
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '01.02.04.02 - Goodwill',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '1.02.04'],
                                    ['description', 'contains_all', 'goodwill']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '01.02.04.02.01 - Goodwill',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.04.02'],
                                            ['description', 'contains_any', ['ágio', 'agio', 'goodwill']]
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '01.02.04.02.02 - Mais Valia',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '1.02.04.02'],
                                            ['description', 'contains_all', 'mais valia']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            }
                        ]
                    }
                ]
            }
        ]
    }
]


In [56]:
section_2_criteria = [
    {
        'target': '02 - Passivo Total',
        'filter': [
            ['account', 'level', 1],
            ['account', 'startswith', '2']
        ],
        'sub_criteria': [
            {
                'target': '02.01 - Passivo Circulante de Curto Prazo',
                'filter': [
                    ['account', 'level', 2],
                    ['account', 'startswith', '2.01'],
                    ['description', 'contains_all', 'circul'],
                    ['description', 'not_contains', 'não']
                ],
                'sub_criteria': [
                    {
                        'target': '02.01.01 - Obrigações Sociais e Trabalhistas de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_all', ['soc', 'trab']]
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.01.01 - Obrigações Sociais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.01'],
                                    ['description', 'contains_all', 'soc']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.01.01.02 - Obrigações Trabalhistas',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.01'],
                                    ['description', 'contains_all', 'trabalh']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.01.02 - Fornecedores de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_all', 'forneced']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.02.01 - Fornecedores Nacionais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.02'],
                                    ['description', 'contains_all', 'nacion']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.01.02.02 - Fornecedores Estrangeiros',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.02'],
                                    ['description', 'contains_all', 'estrang']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.01.03 - Obrigações Fiscais de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_all', 'fisc']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.03.01 - Obrigações Fiscais Federais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.03'],
                                    ['description', 'contains_all', 'feder']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.01.03.01.01 - Imposto de Renda e Contribuição Social a Pagar',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.03.01'],
                                            ['description', 'contains_all', 'renda']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.03.01.02 - Outras Obrigações Fiscais Federais',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.03.01'],
                                            ['description', 'contains_all', 'outras']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.03.01.03 - Tributos Parcelados',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.03.01'],
                                            ['description', 'contains_all', 'parcelados']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.01.03.02 - Obrigações Fiscais Estaduais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.03'],
                                    ['description', 'contains_all', 'estad']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.01.03.03 - Obrigações Fiscais Municipais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.03'],
                                    ['description', 'contains_all', 'municip']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.01.04 - Empréstimos e Financiamentos de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_any', ['emprést', 'emprest', 'financ']]
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.04.01 - Empréstimos e Financiamentos',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.04'],
                                    ['description', 'contains_any', ['emprést', 'emprest', 'financ']]
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.01.04.01.01 - Em Moeda Nacional',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.04.01'],
                                            ['description', 'contains_all', 'nacion']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.04.01.02 - Em Moeda Estrangeira',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.04.01'],
                                            ['description', 'contains_all', 'estrang']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.01.04.02 - Debêntures',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.04'],
                                    ['description', 'contains_any', ['debênt', 'debent']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.01.04.03 - Financiamento por Arrendamento Financeiro',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.04'],
                                    ['description', 'contains_all', 'arrend']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.01.05 - Outras Obrigações de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_all', 'obrig'],
                            ['description', 'not_contains', 'fisc']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.05.01 - Passivos com Partes Relacionadas',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.05'],
                                    ['description', 'contains_all', 'passiv']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.01.05.01.01 - Débitos com Coligadas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.01'],
                                            ['description', 'contains_all', 'colig']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.05.01.03 - Débitos com Controladores',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.01'],
                                            ['description', 'contains_all', 'control']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.05.01.04 - Débitos com Outras Partes Relacionadas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.01'],
                                            ['description', 'contains_none', ['colig', 'control']]
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.01.05.02 - Outros',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.05'],
                                    ['description', 'not_contains', 'passiv']
                                ],
                                'sub_criteria': [
                                    # Group 1: Dividendos e Ações
                                    {
                                        'target': '02.01.05.02.01 - Dividendos e Ações',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.02'],
                                            ['description', 'contains_any', ['divid', ' ações']],
                                        ],
                                        'sub_criteria': []
                                    },
                                    # Group 2: Obrigações Tributárias e Autorizações
                                    {
                                        'target': '02.01.05.02.02 - Obrigações Tributárias e Autorizações',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.02'],
                                            ['description', 'contains_any', ['tribut', 'autoriz', 'concessão']],
                                        ],
                                        'sub_criteria': []
                                    },
                                    # Group 3: Telecomunicações e Consignações
                                    {
                                        'target': '02.01.05.02.03 - Telecomunicações e Consignações',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.02'],
                                            ['description', 'contains_any', ['telecomunicação', 'interconex', 'consign']],
                                        ],
                                        'sub_criteria': []
                                    },
                                    # Group 4: Derivativos e Participações
                                    {
                                        'target': '02.01.05.02.04 - Derivativos e Participações',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.02'],
                                            ['description', 'contains_any', ['derivativ', 'participações']],
                                        ],
                                        'sub_criteria': []
                                    },
                                    # 02.01.05.02.09 - Outros
                                    {
                                        'target': '02.01.05.02.09 - Outros',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.05.02'],
                                            ['description', 'contains_none', ['divid', ' ações', 'tribut', 'autoriz', 'telecomunicação', 'interconex', 'consign', 'derivativ', 'participações']],
                                        ],
                                        'sub_criteria': []
                                    },
                                ]
                            }
                        ]
                    },
                    {
                        'target': '02.01.06 - Provisões de Curto Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.01'],
                            ['description', 'contains_all', 'provis']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.01.06.01 - Provisões Judiciais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.06'],
                                    ['description', 'not_contains', 'outr']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.01.06.01.01 - Provisões Fiscais',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.01'],
                                            ['description', 'contains_all', 'fisc']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.06.01.02 - Provisões Previdenciárias e Trabalhistas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.01'],
                                            ['description', 'contains_any', ['previd', 'trabalh']]
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.06.01.03 - Provisões para Benefícios a Empregados',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.01'],
                                            ['description', 'contains_all', 'benef']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.06.01.04 - Provisões Cíveis',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.01'],
                                            ['description', 'contains_all', 'cív']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.01.06.02 - Outras Provisões',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.01.06'],
                                    ['description', 'contains_all', 'outr']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.01.06.02.01 - Provisões para Garantias',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.02'],
                                            ['description', 'contains_all', 'garant']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.06.02.02 - Provisões para Reestruturação',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.02'],
                                            ['description', 'contains_all', 'reestrut']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.01.06.02.03 - Provisões para Passivos Ambientais e de Desativação',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.01.06.02'],
                                            ['description', 'contains_all', 'ambient']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            }
                        ]
                    }
                ]
            },
            {
                'target': '02.02 - Passivo Não Circulante de Longo Prazo',
                'filter': [
                    ['account', 'level', 2],
                    ['account', 'startswith', '2.02'],
                    ['description', 'contains_all', ['circul', 'não']]
                ],
                'sub_criteria': [
                    {
                        'target': '02.02.01 - Empréstimos e Financiamentos de Longo Prazo',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.02'],
                            ['description', 'contains_any', ['emprést', 'emprest', 'financ']]
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.02.01.01 - Empréstimos e Financiamentos',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.01'],
                                    ['description', 'contains_any', ['emprést', 'emprest', 'financ']]
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.02.01.01.01 - Em Moeda Nacional',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.01.01'],
                                            ['description', 'contains_all', 'nacion']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.01.01.02 - Em Moeda Estrangeira',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.01.01'],
                                            ['description', 'contains_all', 'estrang']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.02.01.02 - Debêntures',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.01'],
                                    ['description', 'contains_any', ['debênt', 'debent']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.02.01.03 - Financiamento por Arrendamento Financeiro',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.01'],
                                    ['description', 'contains_all', 'arrend']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.02.02 - Passivos com Partes Relacionadas de Longo Prazo',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '2.02.02'],
                            ['description', 'contains_all', 'passiv']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.02.02.01 - Débitos com Partes Relacionadas',
                                'filter': [
                                    ['account', 'level', 5],
                                    ['account', 'startswith', '2.02.02'],
                                    ['description', 'contains_all', 'relacion']
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.02.02.01.01 - Débitos com Coligadas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.02'],
                                            ['description', 'contains_all', 'colig']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.02.01.03 - Débitos com Controladores',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.02'],
                                            ['description', 'contains_all', 'control']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.02.01.04 - Débitos com Outras Partes Relacionadas',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.02'],
                                            ['description', 'contains_all', 'relacion']
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            },
                            {
                                'target': '02.02.02.02 - Obrigações por Pagamentos Baseados em Ações',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.02'],
                                    ['description', 'contains_all', 'ações']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.02.02.03 - Adiantamento para Futuro Aumento de Capital',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.02'],
                                    ['description', 'contains_all', 'capit']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.02.02.04 - Tributos Parcelados',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.02'],
                                    ['description', 'contains_all', 'tribut']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.02.03 - Imposto de Renda e Contribuição Social Diferidos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '2.02.03'],
                            ['description', 'contains_all', 'renda']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '02.02.04 - Provisões de Longo Prazo',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '2.02.04'],
                            ['description', 'contains_any', ['provis']], 
                            ['description', 'contains_none', ['emprest', 'emprést', 'debent', 'debênt', 'outr', 'renda']]
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.02.04.01 - Provisões Fiscais Previdenciárias Trabalhistas e Cíveis',
                                'filter': [
                                    ['account', 'level', 5],
                                    ['account', 'startswith', '2.02.04.01'],
                                    ['description', 'contains_any', ['fisc', 'previd', 'trabalh', 'benef', 'cív']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.02.04.02 - Outras Provisões',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.02.04'],
                                    ['description', 'contains_none', ['fisc', 'previd', 'trabalh', 'benef', 'cív']]
                                ],
                                'sub_criteria': [
                                    {
                                        'target': '02.02.04.02.01 - Provisões para Garantias',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.04'],
                                            ['description', 'contains_any', 'garant']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.04.02.02 - Provisões para Reestruturação',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.04'],
                                            ['description', 'contains_any', 'reestrutur']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.04.02.03 - Provisões para Passivos Ambientais e de Desativação',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.04'],
                                            ['description', 'contains_any', 'ambient']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.04.02.04 - Fornecedores de Equipamentos',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.04'],
                                            ['description', 'contains_any', 'forneced']
                                        ],
                                        'sub_criteria': []
                                    },
                                    {
                                        'target': '02.02.04.02.09 - Outras Obrigações',
                                        'filter': [
                                            ['account', 'level', 5],
                                            ['account', 'startswith', '2.02.06'],
                                            ['description', 'contains_none', ['garant', 'reestrutur', 'ambient', 'forneced']]
                                        ],
                                        'sub_criteria': []
                                    }
                                ]
                            }
                        ]
                    }
                ]
            },
            {
                'target': '02.03 - Patrimônio Líquido',
                'filter': [
                    ['account', 'level', 2],
                    ['account', 'startswith', '2.03'],
                    ['description', 'contains_all', ['patrim', 'líquid']]
                ],
                'sub_criteria': [
                    {
                        'target': '02.03.01 - Capital Social Realizado',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'soc']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.03.01.01 - Capital Social',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.01'],
                                    ['description', 'contains_all', 'social']
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.03.01.02 - Gastos na emissão de ações',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.01'],
                                    ['description', 'contains_all', 'ações']
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.03.02 - Reservas de Capital',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03.02'],
                            ['description', 'contains_all', 'capit']
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.03.02.01 - Ágio e Reserva Especial',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.02'],
                                    ['description', 'contains_any', ['ágio', 'prêm', 'prem', 'reserv']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.03.02.02 - Ações, Remuneração e Opções',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.02'],
                                    ['description', 'contains_any', [' ações', ' opções', 'remunera', 'tesour']], 
                                    ['description', 'contains_none', ['ágio', 'prêm', 'prem', 'reserv']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.03.02.09 - Outros',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.02'],
                                    ['description', 'contains_none', ['ágio', 'prêm', 'prem', 'reserv', ' ações', ' opções', 'remunera', 'tesour']]
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.03.03 - Reservas de Reavaliação',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'reaval']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '02.03.04 - Reservas de Lucros',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'lucr']
                        ],
                        'sub_criteria': [
                            # Group 1: Legal and Statutory Reserves
                            {
                                'target': '02.03.04.01 - Reservas Legais e Estatutárias',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.04'],
                                    ['description', 'contains_any', ['legal', 'estatutár']]
                                ],
                                'sub_criteria': []
                            },
                            # Group 2: Retenção e Incentivos Fiscais
                            {
                                'target': '02.03.04.02 - Retenção de Lucros e Incentivos Fiscais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.04'],
                                    ['description', 'contains_any', ['retenção', 'incentivo', 'expansão', 'modernização']]
                                ],
                                'sub_criteria': []
                            },
                            # Group 3: Dividendos e Ações em Tesouraria
                            {
                                'target': '02.03.04.03 - Dividendos e Ações em Tesouraria',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.04'],
                                    ['description', 'contains_any', ['dividendo', 'ações em tesouraria']]
                                ],
                                'sub_criteria': []
                            },
                            # Group 4: Outros
                            {
                                'target': '02.03.04.09 - Outros',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.04'],
                                    ['description', 'contains_none', ['lucr', 'legal', 'estatutár', 'retenção', 'incentivo', 'expansão', 'modernização', 'dividendo', 'ações em tesouraria']]
                                ],
                                'sub_criteria': []
                            }
                        ]
                    }, 
                    {
                        'target': '02.03.05 - Lucros/Prejuízos Acumulados',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'acumul'],
                            ['description', 'not_contains', 'reserv']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '02.03.06 - Ajustes de Avaliação Patrimonial',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', ['ajust', 'patrim']]
                        ],
                        'sub_criteria': [
                            {
                                'target': '02.03.06.01 - Ajustes Patrimoniais',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.06'],
                                    ['description', 'contains_any', ['ajuste', 'custo atribuído']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.03.06.02 - Perdas e Aquisições com Não Controladores',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.06'],
                                    ['description', 'contains_any', ['perda', 'aquisição', 'não controladores']]
                                ],
                                'sub_criteria': []
                            },
                            {
                                'target': '02.03.06.09 - Outros',
                                'filter': [
                                    ['account', 'level', 4],
                                    ['account', 'startswith', '2.03.06'],
                                    ['description', 'contains_none', ['ajuste', 'custo atribuído', 'perda', 'aquisição', 'não controladores']]
                                ],
                                'sub_criteria': []
                            }
                        ]
                    },
                    {
                        'target': '02.03.07 - Ajustes Acumulados de Conversão',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', ['ajust', 'convers']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '02.03.08 - Outros Resultados Abrangentes',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'outr']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '02.03.09 - Participação dos Acionistas Não Controladores',
                        'filter': [
                            ['account', 'level', 3],
                            ['account', 'startswith', '2.03'],
                            ['description', 'contains_all', 'controlad']
                        ],
                        'sub_criteria': []
                    }
                ]
            }
        ]
    }
]


In [55]:
section_3_criteria = [
    {
        'target': '03.01 - Receita de Venda de Bens e/ou Serviços',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.01'],
            ['description', 'contains_all', ['venda', 'bens']]
        ],
        'sub_criteria': []
    },
    {
        'target': '03.02 - Custo dos Bens e/ou Serviços Vendidos',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.02'],
            ['description', 'contains_all', ['custo']]
        ],
        'sub_criteria': []
    },
    {
        'target': '03.03 - Resultado Bruto',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.03'],
            ['description', 'contains_all', ['bruto']]
        ],
        'sub_criteria': []
    },
    {
        'target': '03.04 - Despesas/Receitas Operacionais',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.04'],
            ['description', 'contains_all', ['despesas', 'receitas']]
        ],
        'sub_criteria': [
            {
                'target': '03.04.01 - Despesas Operacionais',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '3.04.01']
                ],
                'sub_criteria': [
                    {
                        'target': '03.04.01.01 - Despesas Comerciais',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.01'],
                            ['description', 'contains_any', 'comerci']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.02 - Despesas Gerais e Administrativas',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.02'],
                            ['description', 'contains_all', 'administrativ']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.03 - Honorários da Diretoria',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.03'],
                            ['description', 'contains_all', 'diretor']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.04 - Despesas Tributárias e Judiciais',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.04'],
                            ['description', 'contains_any', ['tribut', 'judic', 'impost']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.05 - Despesas com Pessoal e Encargos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.05'],
                            ['description', 'contains_all', 'pessoal']
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.06 - Dividendos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.07'],
                            ['description', 'contains_any', ['dividend']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.07 - Provisões',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.08'],
                            ['description', 'contains_any', ['provis']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.08 - Diversos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.06'],
                            ['description', 'contains_none', ['comerci', 'administrativ', 'diretor', 'tribut', 'judic', 'impost', 'pessoal', 'dividend', 'provis', 'outr']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '03.04.01.09 - Outras Despesas',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '3.04.01.09'],
                            ['description', 'contains_any', ['outr']]
                        ],
                        'sub_criteria': []
                    }
                ]
            }
        ]
    },
    {
        'target': '03.05 - Resultado Antes do Resultado Financeiro e dos Tributos',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.05'],
            ['description', 'contains_all', ['resultado', 'antes', 'financeiro']]
        ],
        'sub_criteria': []
    },
    {
        'target': '03.06 - Resultado Financeiro',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.06'],
            ['description', 'contains_all', 'financeiro'], 
            ['description', 'contains_none', 'antes']
        ],
        'sub_criteria': []
    },
    {
        'target': '03.07 - Resultado Antes dos Tributos sobre o Lucro',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.07'],
            ['description', 'contains_all', 'tributos'],
            ['description', 'contains_none', 'financeiro']
        ],
        'sub_criteria': []
    },
    {
        'target': '03.08 - Imposto de Renda e Contribuição Social sobre o Lucro',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.08'],
            ['description', 'contains_all', 'imposto de renda']
        ],
        'sub_criteria': []
    },
    {
        'target': '03.09 - Resultado Líquido das Operações Continuadas',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.09'],
            ['description', 'contains_all', ['resultado líquido', 'Continuadas']],
            ['description', 'contains_none', 'Descontinuadas']
        ],
        'sub_criteria': []
    }, 
    {
        'target': '03.10 - Resultado Líquido das Operações Descontinuadas',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.10'],
            ['description', 'contains_all', ['resultado líquido', 'Descontinuadas']],
            ['description', 'contains_none', 'Continuadas']
        ],
        'sub_criteria': []
    }, 
    {
        'target': '03.11 - Lucro do Período',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '3.11'],
            ['description', 'contains_all', ['lucro', 'período']]
        ],
        'sub_criteria': [
            {
                'target': '03.11.01 - Atribuído a Sócios da Empresa Controladora',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '3.11.01'],
                    ['description', 'contains_all', 'controladora'], 
                    ['description', 'contains_none', 'não']
                ],
                'sub_criteria': []
            },
            {
                'target': '03.11.02 - Atribuído a Sócios Não Controladores',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '3.11.02'],
                    ['description', 'contains_all', ['não', 'controladores']], 
                ],
                'sub_criteria': []
            }
        ]
    }
]


In [65]:
section_6_criteria = [
    {
        'target': '06.01 - Caixa de Operações (Operacional)',
        'filter': [
            ['account', 'equals', '6.01'],
       ],
    },
    {
        'target': '06.02 - Caixa de Investimento',
        'filter': [
            ['account', 'equals', '6.02'],
       ],
    },
    {
        'target': '06.03 - Caixa de Financiamento',
        'filter': [
            ['account', 'equals', '6.03'],
       ],
    },
]

In [59]:
section_7_criteria = [
    {
        'target': '07.01 - Receitas',
        'filter': [['account', 'equals', '7.01']],
        'sub_criteria': [
            {
                'target': '07.01.01 - Vendas de Mercadorias, Produtos e Serviços',
                'filter': [['account', 'equals', '7.01.01']],
                'sub_criteria': []
            },
            {
                'target': '07.01.02 - Outras Receitas',
                'filter': [['account', 'equals', '7.01.02']],
                'sub_criteria': []
            },
            {
                'target': '07.01.03 - Receitas refs. à Construção de Ativos Próprios',
                'filter': [['account', 'equals', '7.01.03']],
                'sub_criteria': []
            },
            {
                'target': '07.01.04 - Provisão/Reversão de Créds. Liquidação Duvidosa',
                'filter': [['account', 'equals', '7.01.04']],
                'sub_criteria': []
            }
        ]
    },
    {
        'target': '07.02 - Insumos Adquiridos de Terceiros',
        'filter': [['account', 'equals', '7.02']],
        'sub_criteria': [
            {
                'target': '07.02.01 - Custos Prods., Mercs. e Servs. Vendidos',
                'filter': [['account', 'equals', '7.02.01']],
                'sub_criteria': []
            },
            {
                'target': '07.02.02 - Materiais, Energia, Servs. de Terceiros e Outros',
                'filter': [['account', 'equals', '7.02.02']],
                'sub_criteria': []
            },
            {
                'target': '07.02.03 - Perda/Recuperação de Valores Ativos',
                'filter': [['account', 'equals', '7.02.03']],
                'sub_criteria': []
            },
            {
                'target': '07.02.04 - Outros',
                'filter': [['account', 'equals', '7.02.04']],
                'sub_criteria': []
            }
        ]
    },
    {
        'target': '07.03 - Valor Adicionado Bruto',
        'filter': [['account', 'equals', '7.03']],
        'sub_criteria': []
    },
    {
        'target': '07.04 - Retenções',
        'filter': [['account', 'equals', '7.04']],
        'sub_criteria': [
            {
                'target': '07.04.01 - Depreciação, Amortização e Exaustão',
                'filter': [['account', 'equals', '7.04.01']],
                'sub_criteria': []
            },
            {
                'target': '07.04.02 - Outras',
                'filter': [['account', 'equals', '7.04.02']],
                'sub_criteria': []
            }
        ]
    },
    {
        'target': '07.05 - Valor Adicionado Líquido Produzido',
        'filter': [['account', 'equals', '7.05']],
        'sub_criteria': []
    },
    {
        'target': '07.06 - Vlr Adicionado Recebido em Transferência',
        'filter': [['account', 'equals', '7.06']],
        'sub_criteria': [
            {
                'target': '07.06.01 - Resultado de Equivalência Patrimonial',
                'filter': [['account', 'equals', '7.06.01']],
                'sub_criteria': []
            },
            {
                'target': '07.06.02 - Receitas Financeiras',
                'filter': [['account', 'equals', '7.06.02']],
                'sub_criteria': []
            },
            {
                'target': '07.06.03 - Outros',
                'filter': [['account', 'equals', '7.06.03']],
                'sub_criteria': [
                    {
                        'target': '07.06.03.01 - Dividendos',
                        'filter': [
                            ['account', 'startswith', '7.06.03'],
                            ['description', 'contains_all', ['dividend']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.06.03.02 - Aluguéis',
                        'filter': [
                            ['account', 'startswith', '7.06.03'],
                            ['description', 'contains_all', ['alugue']]
                        ],
                        'sub_criteria': []
                    }
                ]
            }
        ]
    },
    {
        'target': '07.07 - Valor Adicionado Total a Distribuir',
        'filter': [['account', 'equals', '7.07']],
        'sub_criteria': []
    },
    {
        'target': '07.08 - Distribuição do Valor Adicionado',
        'filter': [['account', 'equals', '7.08']],
        'sub_criteria': [
            {
                'target': '07.08.01 - Pessoal',
                'filter': [['account', 'equals', '7.08.01']],
                'sub_criteria': [
                    {
                        'target': '07.08.01.01 - Remuneração Direta',
                        'filter': [['account', 'equals', '7.08.01.01']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.01.02 - Benefícios',
                        'filter': [['account', 'equals', '7.08.01.02']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.01.03 - F.G.T.S.',
                        'filter': [['account', 'equals', '7.08.01.03']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.01.04 - Outros',
                        'filter': [['account', 'equals', '7.08.01.04']],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '07.08.02 - Impostos, Taxas e Contribuições',
                'filter': [['account', 'equals', '7.08.02']],
                'sub_criteria': [
                    {
                        'target': '07.08.02.01 - Federais',
                        'filter': [['account', 'equals', '7.08.02.01']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.02.02 - Estaduais',
                        'filter': [['account', 'equals', '7.08.02.02']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.02.03 - Municipais',
                        'filter': [['account', 'equals', '7.08.02.03']],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '07.08.03 - Remuneração de Capitais de Terceiros',
                'filter': [['account', 'equals', '7.08.03']],
                'sub_criteria': [
                    {
                        'target': '07.08.03.01 - Juros',
                        'filter': [['account', 'equals', '7.08.03.01']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.03.02 - Aluguéis',
                        'filter': [['account', 'equals', '7.08.03.02']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.03.03 - Outras',
                        'filter': [['account', 'equals', '7.08.03.03']],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '07.08.04 - Remuneração de Capitais Próprios',
                'filter': [['account', 'equals', '7.08.04']],
                'sub_criteria': [
                    {
                        'target': '07.08.04.01 - Juros sobre o Capital Próprio',
                        'filter': [['account', 'equals', '7.08.04.01']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.04.02 - Dividendos',
                        'filter': [['account', 'equals', '7.08.04.02']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.04.03 - Lucros Retidos / Prejuízo do Período',
                        'filter': [['account', 'equals', '7.08.04.03']],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.04.04 - Part. Não Controladores nos Lucros Retidos',
                        'filter': [['account', 'equals', '7.08.04.04']],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '07.08.05 - Outros',
                'filter': [['account', 'equals', '7.08.05']],
                'sub_criteria': [
                    {
                        'target': '07.08.05.01 - Provisões trabalhistas e cíveis, líquidas',
                        'filter': [
                            ['account', 'startswith', '7.08.05'],
                            ['description', 'contains_all', ['trabalhist']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.05.02 - Investimento Social',
                        'filter': [
                            ['account', 'startswith', '7.08.05'],
                            ['description', 'contains_all', ['social']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.05.03 - Lucros Retidos',
                        'filter': [
                            ['account', 'startswith', '7.08.05'],
                            ['description', 'contains_all', ['lucr']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.05.04 - Participação Minoritária',
                        'filter': [
                            ['account', 'startswith', '7.08.05'],
                            ['description', 'contains_all', ['minorit']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '07.08.05.09 - Outros',
                        'filter': [
                            ['account', 'startswith', '7.08.05'],
                            ['description', 'contains_none', ['trabalhist', 'social', 'lucr', 'minorit']]
                        ],
                        'sub_criteria': []
                    }
                ]
            }
        ]
    }
]


In [64]:
section_6_criteria_o1 = [
    {
        'target': '6.01 - Fluxo de Caixa das Atividades Operacionais',
        'filter': [
            ['account', 'level', 2],
            ['account', 'startswith', '6.01']
        ],
        'sub_criteria': [
            {
                'target': '6.01.01 - Fluxo de Caixa das Operações Continuadas',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '6.01.01']
                ],
                'sub_criteria': [
                    {
                        'target': '6.01.01.01 - Provisões e Reversões',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['provisão', 'provisões', 'reversão', 'reversões', 'constituição de provisão', 'provisões pagas']],
                            ['description', 'not_contains_any', ['depreciação', 'amortização', 'perda estimada', 'crédito de liquidação duvidosa', 'despesa financeira']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.02 - Despesas Financeiras',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['despesa de juros', 'encargos sobre empréstimos', 'juros sobre empréstimos', 'juros incorridos', 'amortização do custo de captação', 'apropriação de comissão', 'despesas financeiras']],
                            ['description', 'not_contains_any', ['receita', 'rendimento', 'ganho', 'dividendos', 'atualização monetária']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.03 - Receitas Financeiras',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['receita', 'rendimento', 'juros sobre aplicações financeiras', 'dividendos', 'atualização de crédito tributário', 'ganho', 'ganhos']],
                            ['description', 'not_contains_any', ['despesa', 'encargo', 'perda', 'amortização', 'perdas']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.04 - Perdas em Créditos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['perda estimada', 'perdas estimadas', 'perdas por créditos', 'perdas esperadas', 'redução ao valor recuperável', 'créditos de liquidação duvidosa', 'perdas com crédito']],
                            ['description', 'not_contains_any', ['baixa de ativo', 'depreciação', 'amortização', 'alienação']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.05 - Baixa e Alienação de Ativos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['baixa de ativo', 'baixa de ativos', 'baixa de imobilizado', 'baixa de intangível', 'perda na baixa de ativo', 'alienação de ativos', 'resultado na baixa de bens', 'perda na baixa de ativo permanente']],
                            ['description', 'not_contains_any', ['perda estimada', 'depreciação', 'amortização', 'créditos de liquidação duvidosa', 'provisão']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.06 - Benefícios a Empregados e Incentivos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['plano de opções de ações', 'planos de incentivo', 'plano de opção de ações', 'opção de ações', 'incentivo a longo prazo', 'participação de empregados', 'participação de administradores', 'plano de incentivo', 'pagamento baseado em ações']],
                            ['description', 'not_contains_any', ['provisão', 'perda', 'despesa financeira', 'tributos']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.07 - Imposto de Renda e Contribuição Social',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['imposto de renda', 'contribuição social', 'irpj', 'csll', 'recuperação de tributos', 'crédito tributário', 'tributos diferidos', 'tributos a recuperar', 'impostos', 'taxas', 'contribuições']],
                            ['description', 'not_contains_any', ['taxa de prorrogação', 'icms', 'iss', 'refinanciamento fiscal']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.08 - Depreciação e Amortização',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['depreciação', 'amortização', 'amortização de mais valia', 'amortização de custos de captação', 'amortização direito de uso']],
                            ['description', 'not_contains_any', ['provisão', 'perda', 'receita', 'atualização']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.09 - Resultado de Equivalência Patrimonial',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['equivalência patrimonial', 'resultado de equivalência patrimonial']],
                            ['description', 'not_contains_any', ['perda', 'depreciação', 'amortização', 'provisão']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.10 - Atualizações Monetárias e Variações Cambiais',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['atualização monetária', 'variações monetárias', 'variação cambial', 'atualização financeira', 'variações monetárias sobre perdas judiciais', 'variações monetárias e cambiais']],
                            ['description', 'not_contains_any', ['receita', 'despesa', 'perda estimada', 'créditos de liquidação duvidosa', 'provisão']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.01.11 - Outras Receitas e Despesas Operacionais',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.01'],
                            ['description', 'contains_any', ['outras receitas', 'outras despesas', 'resultado com alienações', 'receita na alienação de bens', 'resultado de operações com sócios não controladores', 'ganho na venda de investimentos', 'ganho na baixa de passivo']],
                            ['description', 'not_contains_any', ['provisão', 'perda', 'depreciação', 'amortização', 'equivalência patrimonial', 'imposto']]
                        ],
                        'sub_criteria': []
                    }
                ]
            },
            {
                'target': '6.01.02 - Variações nos Ativos e Passivos',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '6.01.02']
                ],
                'sub_criteria': [
                    # Subcritérios para 6.01.02.x.y podem ser adicionados aqui
                    # Exemplos:
                    {
                        'target': '6.01.02.01 - Variação em Contas a Receber',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.02'],
                            ['description', 'contains_any', ['contas a receber', 'aumento em contas a receber', 'redução em contas a receber']],
                            ['description', 'not_contains_any', ['estoques', 'impostos', 'fornecedores']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.02.02 - Variação em Estoques',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.02'],
                            ['description', 'contains_any', ['estoques', 'aumento em estoques', 'redução em estoques']],
                            ['description', 'not_contains_any', ['contas a receber', 'impostos', 'fornecedores']]
                        ],
                        'sub_criteria': []
                    },
                    # Outros subcritérios podem ser definidos conforme necessário
                ]
            },
            {
                'target': '6.01.03 - Outros Fluxos de Caixa Operacionais',
                'filter': [
                    ['account', 'level', 3],
                    ['account', 'startswith', '6.01.03']
                ],
                'sub_criteria': [
                    {
                        'target': '6.01.03.01 - Pagamentos de Impostos e Encargos',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.03'],
                            ['description', 'contains_any', ['imposto de renda e contribuição social pagos', 'pagamento de impostos', 'pagamento de irpj', 'pagamento de csll', 'encargos financeiros pagos']],
                            ['description', 'not_contains_any', ['recebimento', 'receita', 'juros recebidos']]
                        ],
                        'sub_criteria': []
                    },
                    {
                        'target': '6.01.03.02 - Pagamentos de Juros',
                        'filter': [
                            ['account', 'level', 4],
                            ['account', 'startswith', '6.01.03'],
                            ['description', 'contains_any', ['juros pagos', 'pagamento de juros', 'encargos financeiros pagos', 'juros sobre empréstimos e financiamentos', 'juros sobre arrendamentos']],
                            ['description', 'not_contains_any', ['recebimento', 'receita', 'juros recebidos']]
                        ],
                        'sub_criteria': []
                    },
                    # Outros subcritérios podem ser adicionados conforme necessário
                ]
            }
        ]
    }
]


In [ ]:
base_name = 'section_7_criteria'
file_path = base_name + '.txt'

# Remove the file if it exists
if os.path.exists(file_path):
    os.remove(file_path)

# Load the DataFrame
sector = 'COMUNICACOES'
df = pd.read_csv(f'..\..\df_{sector}.csv')

# Apply the criteria tree
df = apply_criteria_tree(df, section_0_criteria, output_file=file_path)
df = apply_criteria_tree(df, section_1_criteria, output_file=file_path)
df = apply_criteria_tree(df, section_2_criteria, output_file=file_path)
df = apply_criteria_tree(df, section_3_criteria, output_file=file_path)
df = apply_criteria_tree(df, section_6_criteria, output_file=file_path)
df = apply_criteria_tree(df, section_7_criteria, output_file=file_path)

# Save the DataFrame with the applied criteria
df.to_csv(f'df_standart_{sector}.csv', index=False)


### Financial Ratios

In [2]:
def duplicate_dfs_types(df, source_type='Dados da Empresa', target_types=['DFs Consolidadas', 'DFs Individuais']):
    """
    Duplica condicionalmente as linhas de um tipo específico para outros tipos, 
    baseando-se na existência prévia desses tipos para a mesma empresa e quarter.
    
    Parâmetros:
    - df (pd.DataFrame): DataFrame original contendo os dados financeiros.
    - source_type (str): O tipo de linha que será duplicado. Padrão: 'Dados da Empresa'.
    - target_types (list of str): Lista dos tipos para os quais as linhas serão duplicadas.
                                   Padrão: ['DFs Consolidadas', 'DFs Individuais'].
    
    Retorna:
    - pd.DataFrame: DataFrame atualizado com as duplicações condicionais.
    """
    
    # Passo 1: Identificar combinações existentes de company_name e quarter para cada target_type
    existing_combinations = {}
    for target in target_types:
        existing_keys = df[df['type'] == target][['company_name', 'quarter']].drop_duplicates()
        existing_combinations[target] = existing_keys

    # Passo 2: Filtrar as linhas do source_type
    source_df = df[df['type'] == source_type].copy()
    
    # Passo 3: Para cada target_type, verificar onde duplicar
    duplicated_dfs = []
    for target in target_types:
        # Obter as combinações onde já existe o target_type
        target_keys = existing_combinations[target]
        
        # Realizar um merge para identificar quais linhas do source_df têm a combinação existente
        to_duplicate = source_df.merge(target_keys, on=['company_name', 'quarter'], how='inner', suffixes=('', '_target'))
        
        if not to_duplicate.empty:
            # Duplicar as linhas e alterar o type para o target_type
            duplicated = to_duplicate.copy()
            duplicated['type'] = target
            duplicated_dfs.append(duplicated)
            # print(f"Duplicando {len(duplicated)} linhas para o tipo '{target}'.")
        else:
            # print(f"Nenhuma duplicação necessária para o tipo '{target}'.")
            pass
    
    # Passo 4: Concatenar todas as duplicações
    if duplicated_dfs:
        duplicated_df = pd.concat(duplicated_dfs, ignore_index=True)
    else:
        duplicated_df = pd.DataFrame(columns=df.columns)
        # print("Nenhuma duplicação realizada.")
    
    # Passo 5: Remover as linhas originais do source_type
    df_filtered = df[df['type'] != source_type].copy()
    
    # Passo 6: Adicionar as duplicações ao DataFrame
    if not duplicated_df.empty:
        df_updated = pd.concat([df_filtered, duplicated_df], ignore_index=True)
    else:
        df_updated = df_filtered.copy()
    
    # Passo 7: Resetar o índice e ordenar o DataFrame (opcional)
    df_updated.reset_index(drop=True, inplace=True)
    
    # Passo 8: Ordenar o DataFrame
    df_updated = df_updated.sort_values(
        by=['sector', 'subsector', 'segment', 'company_name', 'quarter', 'version', 'type', 'account', 'description']
    ).reset_index(drop=True)
    
    return df_updated


In [236]:
def add_indicators(df, frame_name, indicator_list):
    """
    Calcula indicadores financeiros e os adiciona como novas linhas no DataFrame.

    Parâmetros:
    - df (pd.DataFrame): DataFrame original contendo os dados financeiros.
    - indicator_list (list of dict): Lista de dicionários com definições dos indicadores.

    Retorna:
    - pd.DataFrame: DataFrame atualizado com as novas linhas de indicadores.
    """
   # Função para realizar divisão segura
    def safe_division(numerator, denominator):
        with np.errstate(divide='ignore', invalid='ignore'):
            result = numerator / denominator
            result = result.replace([np.inf, -np.inf], np.nan)  # Substitui infinitos por NaN
            return result

    # Passo 1: Pivotar o DataFrame com contas como colunas e incluir 'quarter' no índice
    pivot_df = df.pivot_table(
        index=['company_name', 'type', 'quarter'],
        columns='account',
        values='value',
        aggfunc='sum',
        fill_value=0  # Substitui valores ausentes por 0
    ).reset_index()
    
    # print("Colunas do DataFrame Pivotado:", pivot_df.columns.tolist())
    
    # Passo 2: Calcular os Indicadores
    for indicator in indicator_list:
        column_name = indicator['description']
        column_account = indicator['account']
        try:
            # Aplicar a fórmula para calcular o indicador
            pivot_df[column_name] = indicator['formula'](pivot_df)
            # print(f"Indicador '{column_name}' calculado com sucesso.")
        except KeyError as e:
            print(f"'{column_account} - {column_name}': a conta {e} não existe")
            pivot_df[column_name] = np.nan  # Atribuir NaN se a coluna não existir
        except Exception as e:
            print(f"{column_name}': {e} Erro ao calcular o indicador '{column_name}': {e}")
            pivot_df[column_name] = np.nan  # Atribuir NaN em caso de erro

    # Passo 3: Criar Novas Linhas para os Indicadores
    new_rows = []
    
    for indicator in indicator_list:
        account = indicator['account']
        description = indicator['description']
        
        if description not in pivot_df.columns:
            print(f"Aviso: Indicador '{description}' não foi calculado e será ignorado.")
            continue
        
        # Extrair os valores calculados do indicador
        pivot_df['value'] = pivot_df[description]
        
        # Selecionar as colunas relevantes
        indicator_values = pivot_df[['company_name', 'type', 'quarter', 'value']].copy()
        indicator_values['account'] = account
        indicator_values['description'] = description
        
        # Atribuir 'frame' como 'Indicadores' para identificar facilmente as novas linhas
        indicator_values['frame'] = frame_name
        
        # Preencher as colunas faltantes com informações do DataFrame original
        # Selecionar combinações únicas de 'company_name', 'type' e 'quarter'
        metadata = df[['company_name', 'type', 'quarter', 'nsd', 'sector', 'subsector', 'segment', 'version']].drop_duplicates(subset=['company_name', 'type', 'quarter'], keep='first')
        
        # Mesclar com 'metadata' para preencher 'nsd', 'sector', etc.
        indicator_row = indicator_values.merge(metadata, on=['company_name', 'type', 'quarter'], how='left')
        
        # Reordenar as colunas para corresponder ao DataFrame original
        indicator_row = indicator_row[['nsd', 'sector', 'subsector', 'segment', 'company_name', 'quarter', 'version', 'type', 'frame', 'account', 'description', 'value']]
        
        # Adicionar à lista de novas linhas
        new_rows.append(indicator_row)
    
    # Combinar todas as novas linhas de indicadores
    if new_rows:
        new_rows_df = pd.concat(new_rows, ignore_index=True)
    else:
        new_rows_df = pd.DataFrame(columns=df.columns)
        # print("Nenhum indicador foi adicionado.")
    
    # Passo 4: Adicionar as Novas Linhas ao DataFrame Original
    updated_df = pd.concat([df, new_rows_df], ignore_index=True)
    
    # Passo 5: Tratamento Final dos Valores
    # Opcional: Preencher valores faltantes ou realizar outras limpezas
    # Exemplo: Substituir NaN por zero onde apropriado
    # updated_df['value'] = updated_df['value'].fillna(0)
    
    return updated_df


In [ ]:
sector = 'COMUNICACOES'
db_file = f"..\data\{settings.db_name.split('.')[0]} {settings.statements_standard}.db"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)
# df['account_description'] = df['account'] + ' - ' + df['description']
df['quarter'] = pd.to_datetime(df['quarter'])

# df = clean_dados_da_empresa(df)
df = duplicate_dfs_types(df)

# Store unique values
uv = {}
for col in df.columns:
    if col not in ['nsd', 'value']:
        uv[col] = sorted(df[col].unique().tolist())

df[['account', 'description']].drop_duplicates().to_csv('account-description.csv', index=False)

In [249]:
# Definition of indicator formulas
indicators_08 = [
    {
        'account': '08.01',
        'description': 'ROE Return On Equity',
        'formula': lambda df: df['03.11'] / df['02.03'] * 10000  # ROE = Net Profit / Shareholders' Equity * 100
    },
    {
        'account': '08.02',
        'description': 'Índice de Liquidez Corrente',
        'formula': lambda df: df['01.01'] / df['02.01']  # Current Ratio = Current Assets / Current Liabilities
    },
    # Add more indicators as needed
]


In [255]:
df = add_indicators(df, 'Indicadores Fundamentalistas', indicators_08)
# Store unique values
uvi = {}
for col in df.columns:
    if col not in ['nsd', 'value']:
        uvi[col] = sorted(df[col].unique().tolist())


In [ ]:
mask  = df['company_name'] == uvi['company_name'][7]
mask &= df['type'] == uvi['type'][1]
mask &= df['account'] == '08.02'
# mask &= df2['quarter'] == '2020-12-31'
df[mask]

In [ ]:
index_cols = [col for col in df2.columns if col not in ['quarter', 'value']]

# Verificar duplicatas
duplicatas = df2.duplicated(subset=index_cols + ['quarter'], keep=False)
num_duplicatas = duplicatas.sum()

if num_duplicatas > 0:
    print("Agregando duplicatas somando os valores.")
    df2_agregado = df2.groupby(index_cols + ['quarter'], as_index=False)['value'].sum()
    # df2_agregado = df2.groupby(index_cols + ['quarter'], as_index=False)['value'].mean() # opcionalmente calcular a média

else:
    df2_agregado = df2.copy()

# Pivotar o DataFrame
df_pivot = df2_agregado.pivot_table(
    index=index_cols,
    columns='quarter',
    values='value',
    aggfunc='first'  # 'first' assume que não há duplicatas após a agregação
).reset_index()

# # Obter uma lista dos quarters únicos, ordenados de forma decrescente
# sorted_quarters = sorted(df_pivot.columns[ len(index_cols): ], reverse=True)

# # Reordenar as colunas: primeiro os índices, depois os quarters em ordem decrescente
# df_pivot = df_pivot[index_cols + sorted_quarters]

# # Opcional: formatar os nomes das colunas de quarter para strings legíveis
# df_pivot.columns = list(index_cols) + [q.strftime('%Y-%m-%d') for q in sorted_quarters]

df_pivot.fillna(0, inplace=True)
df_pivot


In [ ]:
df_pivot.iloc[0:1,]

### Plots

In [2]:
sector = 'COMUNICACOES'
db_file = f"..\data\{settings.db_name.split('.')[0]} {settings.statements_standard}.db"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM {sector}", conn)
# df['account_description'] = df['account'] + ' - ' + df['description']
df['quarter'] = pd.to_datetime(df['quarter'])


In [4]:
# Store unique values
uv = {}
for col in df.columns:
    if col not in ['nsd', 'value']:
        uv[col] = sorted(df[col].unique().tolist())


In [117]:
def plot_series(company_name, account, start_date=None, end_date=None, window=4, std_dev=2, normalize=True):
    """
    Plot a time series of a financial metric for a given company, with Bollinger Bands and custom blue color scheme.
    
    Args:
    - company_name: Name of the company.
    - account: The financial account to be plotted.
    - start_date: Start date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - window: The window size for the moving average (default is 4).
    - std_dev: The number of standard deviations for the Bollinger Bands (default is 2).
    """
    # Define blue color scheme
    color_blue = {
        'main': 'blue',                
        'mma': 'lightblue',            
        'bands': 'lightblue',          
        'fill': 'rgba(173, 216, 230, 0.2)',  
        'extra_bands': 'lightgray',    
        'extra_fill': 'rgba(200, 200, 200, 0.2)'  
    }

    # Try 'DFs Consolidadas' first
    if (df['type'] == 'DFs Consolidadas').any():
        df_type = 'DFs Consolidadas'
    else:
        df_type = 'DFs Individuais'

    # Filter data
    data = df[
        (df['company_name'] == company_name) & 
        (df['account'] == account) & 
        (df['type'] == df_type)
    ]

    # Apply date filters if provided
    if start_date:
        data = data[data['quarter'] >= pd.to_datetime(start_date)]
    if end_date:
        data = data[data['quarter'] <= pd.to_datetime(end_date)]

    if data.empty:
        print("No data available for the given filters, check parameters.")
        return

    if normalize == True:
        # Drop the first row if its value is zero
        if data['value'].iloc[0] == 0:
            data = data.iloc[1:].copy()

        # Get the first non-zero value
        first_non_zero = data['value'].iloc[0]

        # Normalize the values by dividing by the first non-zero value
        data['value'] = data['value'] / first_non_zero

        # Scale the values so that the maximum is 100
        max_value = data['value'].max()
        data['value'] = (data['value'] / max_value) * 100

    # Sort data by date
    data = data.sort_values('quarter')

    # Calculate the moving average (Middle Band)
    data['mma'] = data['value'].rolling(window=window).mean()

    # Calculate the rolling standard deviation
    data['std_dev'] = data['value'].rolling(window=window).std()

    # Calculate upper and lower Bollinger Bands with std_dev multiplier
    data['upper_band'] = data['mma'] + std_dev * data['std_dev']
    data['lower_band'] = data['mma'] - std_dev * data['std_dev']

    # Calculate extra upper and lower bands using std_dev * 1.5
    extra_std_dev = std_dev * 1.5
    data['upper_band_extra'] = data['mma'] + extra_std_dev * data['std_dev']
    data['lower_band_extra'] = data['mma'] - extra_std_dev * data['std_dev']

    # Set description to the first non-null description value in the filtered data
    description = data['description'].dropna().iloc[0] if 'description' in data.columns and not data['description'].isna().all() else account

    # Create line plot with original values
    fig = px.line(
        data,
        x='quarter',
        y='value',
        title=f'{account} - {description} with Bollinger Bands for {company_name} ({df_type})',
        labels={'quarter': 'Quarter', 'value': description}, 
        template='plotly_white'  
    )
    
    # Add main line for the data
    fig.update_traces(line=dict(color=color_blue['main']), name='Original Data', showlegend=False)

    # Add moving average (MMA) as a dashed light blue line
    fig.add_scatter(
        x=data['quarter'],
        y=data['mma'],
        mode='lines',
        name=f'Middle Band (window={window})',
        line=dict(color=color_blue['mma'], dash='dash'),
        showlegend=False
    )

    # Add upper and lower Bollinger Bands (std_dev * 2) and group them
    fig.add_scatter(
        x=data['quarter'],
        y=data['upper_band'],
        mode='lines',
        name='Upper Band (std_dev * 2)',
        line=dict(dash='solid', color=color_blue['bands']),
        legendgroup='Bands (std_dev * 2)',
        showlegend=False
    )

    fig.add_scatter(
        x=data['quarter'],
        y=data['lower_band'],
        mode='lines',
        name='Lower Band (std_dev * 2)',
        line=dict(dash='solid', color=color_blue['bands']),
        legendgroup='Bands (std_dev * 2)',
        showlegend=False
    )

    # Fill the area between the upper and lower bands (std_dev * 2)
    fig.add_scatter(
        x=data['quarter'],
        y=data['upper_band'],
        fill=None,  
        mode='lines',
        name='Upper Band Fill (std_dev * 2)',
        line=dict(color='rgba(0,0,0,0)'),  
        showlegend=False,
        legendgroup='Bands (std_dev * 2)',
    )

    fig.add_scatter(
        x=data['quarter'],
        y=data['lower_band'],
        fill='tonexty',  
        mode='lines',
        name='Lower Band Fill (std_dev * 2)',
        line=dict(color='rgba(0,0,0,0)'),  
        fillcolor=color_blue['fill'],  
        showlegend=False,
        legendgroup='Bands (std_dev * 2)',
    )

    # Add extra upper and lower Bollinger Bands (std_dev * 1.5) and group them
    fig.add_scatter(
        x=data['quarter'],
        y=data['upper_band_extra'],
        mode='lines',
        name='Upper Band (std_dev * 1.5)',
        line=dict(dash='dot', color=color_blue['extra_bands']),
        legendgroup='Bands (std_dev * 1.5)',
        showlegend=False
    )

    fig.add_scatter(
        x=data['quarter'],
        y=data['lower_band_extra'],
        mode='lines',
        name='Lower Band (std_dev * 1.5)',
        line=dict(dash='dot', color=color_blue['extra_bands']),
        legendgroup='Bands (std_dev * 1.5)',
        showlegend=False
    )

    # Fill the area between the extra upper and lower bands (std_dev * 1.5)
    fig.add_scatter(
        x=data['quarter'],
        y=data['upper_band_extra'],
        fill=None,  
        mode='lines',
        name='Upper Band Extra Fill (std_dev * 1.5)',
        line=dict(color='rgba(0,0,0,0)'),  
        showlegend=False,
        legendgroup='Bands (std_dev * 1.5)',
    )

    fig.add_scatter(
        x=data['quarter'],
        y=data['lower_band_extra'],
        fill='tonexty',  
        mode='lines',
        name='Lower Band Extra Fill (std_dev * 1.5)',
        line=dict(color='rgba(0,0,0,0)'),  
        fillcolor=color_blue['extra_fill'],  
        showlegend=False,
        legendgroup='Bands (std_dev * 1.5)',
    )

    # Update layout
    fig.update_layout(xaxis_title='Quarter', yaxis_title=description)
    
    fig.show()

    return data


In [ ]:
data = plot_series(
    company_name=uv['company_name'][7],  # Change to your desired company
    account='01.01',
    # start_date='2015-01-01',
    # end_date='2023-06-30', 
    window=4, 
    normalize=False
)


In [119]:
def plot_time_series(company_name, accounts, start_date=None, end_date=None, plot_type="stacked"):
    """
    Plot a stacked, overlapping, line-only, 100% stacked, or normalized stacked area chart of financial metrics for multiple accounts of a given company.
    
    Args:
    - company_name: Name of the company.
    - accounts: List of financial accounts to be plotted.
    - start_date: Start date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines the type of plot: "stacked", "overlapping", "lines", "100_stacked", or "normalized_stacked".
    """
    # Filter and combine data for all accounts
    filtered_data = pd.DataFrame()
    
    for account in accounts:
        # Try 'DFs Consolidadas' first
        if (df['type'] == 'DFs Consolidadas').any():
            df_type = 'DFs Consolidadas'
        else:
            df_type = 'DFs Individuais'

        # Filter data for the current account
        data = df[
            (df['company_name'] == company_name) & 
            (df['account'] == account) & 
            (df['type'] == df_type)
        ]
        
        # Apply date filters if provided
        if start_date:
            data = data[data['quarter'] >= pd.to_datetime(start_date)]
        if end_date:
            data = data[data['quarter'] <= pd.to_datetime(end_date)]

        if data.empty:
            print(f"No data available for account {account}. Skipping.")
            continue

        # Apply normalization only when plot_type is "normalized_stacked"
        if plot_type == "normalized_stacked":
            # Drop the first row if its value is zero
            if data['value'].iloc[0] == 0:
                data = data.iloc[1:].copy()

            # Get the first non-zero value and normalize all data by that value
            first_non_zero_value = data['value'][data['value'] != 0].iloc[0]

            # Normalize to ensure the first non-zero value is 1
            data.loc[:, 'value'] = data['value'] / first_non_zero_value

        # Sort data by date
        data = data.sort_values('quarter')
        
        # Add the account as a column for identifying each account
        data['account'] = account

        # Append to the cumulative data
        filtered_data = pd.concat([filtered_data, data])

    if filtered_data.empty:
        print("No data available for the given filters, check parameters.")
        return
    
    # Determine the Y-axis label
    y_axis_title = 'Value'
    
    # Plot logic for different plot types
    if plot_type == "stacked":
        # Stacked area plot with raw values
        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    elif plot_type == "overlapping":
        # Overlapping area plot (using filled line plots to avoid stacking)
        fig = px.line(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Overlapping Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )

        # Add filled areas to create the overlapping effect
        for account in accounts:
            account_data = filtered_data[filtered_data['account'] == account]
            fig.add_scatter(
                x=account_data['quarter'],
                y=account_data['value'],
                mode='lines',
                line=dict(width=0.5),
                fill='tozeroy',
                name=account,
                legendgroup=account,  # Group line and area under the same legend
                showlegend=False  # Hide the second entry for the filled area
            )

    elif plot_type == "lines":
        # Line-only plot with no filling
        fig = px.line(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Line-Only Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )

    elif plot_type == "100_stacked":
        # 100% stacked area plot (scaled to show percentages)
        filtered_data['total_per_quarter'] = filtered_data.groupby('quarter')['value'].transform('sum')
        filtered_data['value'] = (filtered_data['value'] / filtered_data['total_per_quarter']) * 100
        y_axis_title = 'Percentage Contribution (%)'
        
        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'100% Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    elif plot_type == "normalized_stacked":
        # Normalized stacked area plot (no percentage, just normalized values for each account)
        y_axis_title = 'Normalized Value'

        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Normalized Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    # Update layout
    fig.update_layout(xaxis_title='Quarter', yaxis_title=y_axis_title, showlegend=True)
    
    fig.show()

    return filtered_data


In [ ]:
#     - plot_type: Determines the type of plot: "stacked", "lines", "overlapping", "100_stacked", or "normalized_stacked".
# Example usage: Line-only plot
data = plot_time_series(
    company_name=uv['company_name'][7],
    accounts=['01.01.01', '01.01.02', '01.01.03', '01.01.04', ],
    plot_type="stacked",  # Line-only plot
    start_date='2013-12-31',
)

# Example usage: Line-only plot
data = plot_time_series(
    company_name=uv['company_name'][7],
    accounts=['01.01.01', '01.01.02', '01.01.03', '01.01.04', ],
    plot_type="100_stacked",  # Line-only plot
    start_date='2013-12-31',
)


In [ ]:
def plot_time_series(company_name, accounts, start_date=None, end_date=None, plot_type="stacked"):
    """
    Plot a stacked, overlapping, line-only, 100% stacked, or normalized stacked area chart of financial metrics for multiple accounts of a given company.
    
    Args:
    - company_name: Name of the company.
    - accounts: List of financial accounts to be plotted.
    - start_date: Start date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines the type of plot: "stacked", "overlapping", "lines", "100_stacked", or "normalized_stacked".
    """
    # Filter and combine data for all accounts
    filtered_data = pd.DataFrame()
    
    for account in accounts:
        # Try 'DFs Consolidadas' first
        if (df['type'] == 'DFs Consolidadas').any():
            df_type = 'DFs Consolidadas'
        else:
            df_type = 'DFs Individuais'

        # Filter data for the current account
        data = df[
            (df['company_name'] == company_name) & 
            (df['account'] == account) & 
            (df['type'] == df_type)
        ]
        
        # Apply date filters if provided
        if start_date:
            data = data[data['quarter'] >= pd.to_datetime(start_date)]
        if end_date:
            data = data[data['quarter'] <= pd.to_datetime(end_date)]

        if data.empty:
            print(f"No data available for account {account}. Skipping.")
            continue

        # Apply normalization only when plot_type is "normalized_stacked"
        if plot_type == "normalized_stacked":
            # Drop the first row if its value is zero
            if data['value'].iloc[0] == 0:
                data = data.iloc[1:].copy()

            # Get the first non-zero value and normalize all data by that value
            first_non_zero_value = data['value'][data['value'] != 0].iloc[0]

            # Normalize to ensure the first non-zero value is 1
            data.loc[:, 'value'] = data['value'] / first_non_zero_value

        # Sort data by date
        data = data.sort_values('quarter')
        
        # Add the account as a column for identifying each account
        data['account'] = account

        # Append to the cumulative data
        filtered_data = pd.concat([filtered_data, data])

    if filtered_data.empty:
        print("No data available for the given filters, check parameters.")
        return
    
    # Determine the Y-axis label
    y_axis_title = 'Value'
    
    # Plot logic for different plot types
    if plot_type == "stacked":
        # Stacked area plot with raw values
        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    elif plot_type == "overlapping":
        # Overlapping area plot (using filled line plots to avoid stacking)
        fig = px.line(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Overlapping Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )

        # Add filled areas to create the overlapping effect
        for account in accounts:
            account_data = filtered_data[filtered_data['account'] == account]
            fig.add_scatter(
                x=account_data['quarter'],
                y=account_data['value'],
                mode='lines',
                line=dict(width=0.5),
                fill='tozeroy',
                name=account,
                legendgroup=account,  # Group line and area under the same legend
                showlegend=False  # Hide the second entry for the filled area
            )

    elif plot_type == "lines":
        # Line-only plot with no filling
        fig = px.line(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Line-Only Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )

    elif plot_type == "100_stacked":
        # 100% stacked area plot (scaled to show percentages)
        filtered_data['total_per_quarter'] = filtered_data.groupby('quarter')['value'].transform('sum')
        filtered_data['value'] = (filtered_data['value'] / filtered_data['total_per_quarter']) * 100
        y_axis_title = 'Percentage Contribution (%)'
        
        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'100% Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    elif plot_type == "normalized_stacked":
        # Normalized stacked area plot (no percentage, just normalized values for each account)
        y_axis_title = 'Normalized Value'

        fig = px.area(
            filtered_data,
            x='quarter',
            y='value',
            color='account',
            title=f'Normalized Stacked Area Plot of Accounts for {company_name}',
            labels={'quarter': 'Quarter', 'value': y_axis_title},
            template='plotly_white'
        )
    
    # Update layout
    fig.update_layout(xaxis_title='Quarter', yaxis_title=y_axis_title, showlegend=True)
    
    return fig  # Return the figure, not the DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Initialize a subplot figure with 2 rows and 1 column
fig = make_subplots(rows=2, cols=1, subplot_titles=("Stacked Plot", "100% Stacked Plot"))

# Call plot_time_series for the stacked plot and add to the first subplot (row 1)
fig_stacked = plot_time_series(
    company_name=uv['company_name'][7],
    accounts=['01.01.01', '01.01.02', '01.01.03', '01.01.04'],
    plot_type="stacked",
    start_date='2013-12-31',
)

# Add the traces from the stacked plot to the first subplot
for trace in fig_stacked['data']:
    fig.add_trace(trace, row=1, col=1)

# Call plot_time_series for the 100% stacked plot and add to the second subplot (row 2)
fig_100_stacked = plot_time_series(
    company_name=uv['company_name'][7],
    accounts=['01.01.01', '01.01.02', '01.01.03', '01.01.04'],
    plot_type="100_stacked",
    start_date='2013-12-31',
)

# Add the traces from the 100% stacked plot to the second subplot
for trace in fig_100_stacked['data']:
    fig.add_trace(trace, row=2, col=1)

# Update the layout for the entire figure
fig.update_layout(
    title_text="Comparison of Stacked and 100% Stacked Plots",
    height=800,
    showlegend=False  # Hide the legend since it might overlap
)

# Show the plot
fig.show()


In [220]:
import plotly.graph_objects as go
import pandas as pd

def plot_company_comparison(df, companies, account, start_date=None, end_date=None, plot_type="lines"):
    """
    Compare a financial metric (account) across multiple companies, filtered by 'df_type',
    and display the average of the companies.
    
    Args:
    - df: DataFrame containing the financial data.
    - companies: List of company names to compare.
    - account: The financial account to be plotted (e.g., '03.11 - Lucro do Período').
    - start_date: Start date for filtering the data (optional).
    - end_date: End date for filtering the data (optional).
    - plot_type: Determines whether to plot 'lines' or 'normalized'.
    """
    # Try 'DFs Consolidadas' first
    if (df['type'] == 'DFs Consolidadas').any():
        df_type = 'DFs Consolidadas'
    else:
        df_type = 'DFs Individuais'

    # Filter data for the specified companies, account, and df_type
    data = df[
        (df['company_name'].isin(companies)) &
        (df['account'] == account) &
        (df['type'] == df_type)
    ].copy()  # Make a deep copy to avoid SettingWithCopyWarning

    # Apply start and end date filters if provided
    if start_date:
        data = data[data['quarter'] >= pd.to_datetime(start_date)].copy()  # Copy after filtering
    if end_date:
        data = data[data['quarter'] <= pd.to_datetime(end_date)].copy()  # Copy after filtering

    if data.empty:
        print("No data available for the given filters.")
        return

    # Get the description dynamically from the DataFrame based on the 'account' field
    description = data['description'].dropna().iloc[0] if 'description' in data.columns and not data['description'].isna().all() else account

    # Create the figure
    fig = go.Figure()

    # Add line plot for each company (no markers)
    for company in companies:
        company_data = data[data['company_name'] == company].sort_values('quarter').copy()  # Avoid modifying the original DataFrame

        if not company_data.empty:
            if plot_type == "normalized":
                # Try to get the first non-zero value for normalization
                non_zero_values = company_data['value'][company_data['value'] != 0]

                if not non_zero_values.empty:
                    first_non_zero_value = non_zero_values.iloc[0]
                    # Normalize the values: scale from 1 to proportional max_value
                    company_data.loc[:, 'normalized_value'] = company_data['value'] / first_non_zero_value
                else:
                    print(f"Warning: No non-zero values for {company}. Skipping normalization.")
                    company_data.loc[:, 'normalized_value'] = company_data['value']  # No normalization applied

                # Plot normalized lines
                fig.add_trace(go.Scatter(
                    x=company_data['quarter'],
                    y=company_data['normalized_value'],
                    mode='lines',
                    name=f"{company} (normalized)",
                    hoverinfo='x+y'
                ))
            else:
                # Plot regular lines
                fig.add_trace(go.Scatter(
                    x=company_data['quarter'],
                    y=company_data['value'],
                    mode='lines',
                    name=company,
                    hoverinfo='x+y'
                ))

    if plot_type == "normalized":
        # Calculate the average of actual values for each quarter
        average_data = data.groupby('quarter')['value'].mean().reset_index()

        # Normalize the average value (just like individual company values)
        first_non_zero_avg = average_data['value'][average_data['value'] != 0].iloc[0]
        average_data['normalized_value'] = average_data['value'] / first_non_zero_avg

        # Add the average normalized line to the plot
        fig.add_trace(go.Scatter(
            x=average_data['quarter'],
            y=average_data['normalized_value'],
            mode='lines',
            name='Average (normalized)',
            line=dict(dash='dash', color='black'),
            hoverinfo='x+y'
        ))

        y_axis_title = 'Normalized Value (Proportional to 1)'
    else:
        # Calculate the average value for each quarter
        average_data = data.groupby('quarter')['value'].mean().reset_index()

        # Add the average line to the plot
        fig.add_trace(go.Scatter(
            x=average_data['quarter'],
            y=average_data['value'],
            mode='lines',
            name='Average',
            line=dict(dash='dash', color='black'),
            hoverinfo='x+y'
        ))

        y_axis_title = description

    # Update layout
    fig.update_layout(
        title=f'Comparison of {account} - {description}',
        xaxis_title='Quarter',
        yaxis_title=y_axis_title,
        template='plotly_white',
        showlegend=True
    )

    # Show plot
    fig.show()

    return data



In [ ]:
# Example usage:
data = plot_company_comparison(
    df=df,  # Your DataFrame containing the financial data
    companies=['OI SA', 'TELEFONICA BRASIL SA', 'TIM SA'],
    account='01.01.01',  # Example account code for "Lucro do Período"
    start_date='2013-12-31',  # Filter from this date
    end_date='2020-12-31',     # Optional: Filter until this date
    plot_type="lines"  # Choose between "lines" and "normalized"
)
# Example usage:
data = plot_company_comparison(
    df=df,  # Your DataFrame containing the financial data
    companies=['OI SA', 'TELEFONICA BRASIL SA', 'TIM SA'],
    account='03.01',  # Example account code for "Lucro do Período"
    # start_date='2020-12-31',  # Filter from this date
    # end_date='2023-12-31',     # Optional: Filter until this date
    plot_type="normalized"  # Choose between "lines" and "normalized"
)


### World Data

In [3]:
serie = '1'

url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie}/dados?formato=json'
response = requests.get(url)
data = response.json()
data = pd.DataFrame(data)



In [54]:
def get_median_data(df):
    # Remover a informação de fuso horário (timezone) antes de converter para períodos
    df.index = df.index.tz_localize(None)

    # Criar uma coluna de trimestre ('Quarter')
    df['quarter'] = df.index.to_period('Q')

    # Calcular a mediana de 'Adj Close' por trimestre
    median = df.groupby('quarter')['value'].median()
    # mean = df.groupby('quarter')['Adj Close'].mean()

    # Convert last_dates to the actual last date of the quarter
    # Using the 'Q' period information from 'Quarter'
    last_quarter_dates = df['quarter'].map(lambda x: x.end_time)

    # Converter as datas para manter apenas a parte da data (sem tempo)
    last_quarter_dates = last_quarter_dates.dt.date.drop_duplicates()

    # Criar um DataFrame com a última data correta de cada trimestre e a mediana trimestral
    quarterly_summary = pd.DataFrame({
        'quarter': last_quarter_dates.values,  # Drop duplicates to keep one last date per quarter
        'median': median.values, 
    })


    return quarterly_summary

In [80]:
settings.db_name

'b3.db'

In [130]:
# sector = 'COMUNICACOES'
db_file = f"..\data\{settings.db_name}"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch all table names excluding internal SQLite tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()

df = pd.read_sql_query(f"SELECT * FROM company_info", conn)


In [131]:
# Get the unique ticker codes from the 'ticker_codes' column
ticker_codes = df['ticker_codes'].unique()

# Split the comma-separated ticker codes, flatten the list, and append '.SA' if necessary
ticker_list = [f"{code.strip()}.SA" if not code.strip().endswith('.SA') else code.strip() 
               for tickers in ticker_codes for code in tickers.split(',') if code]

# Remove empty strings (if any) and display the result
companies = [code for code in ticker_list if code]
len(companies)

833

In [132]:
# Baixar os dados
last_date = '1900-01-01'
# companies = ['AAPL', 'ITUB3.SA', 'ITUB4.SA', 'GOOG']
data = yf.download(' '.join(companies), start=last_date)

[******************    37%                       ]  307 of 832 completedCould not get exchangeTimezoneName for ticker 'LIPR3.SA' reason: 'chart'
[*********************100%***********************]  831 of 832 completed

245 Failed downloads:
['BAAX39.SA', 'BGOV39.SA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1900-01-01 -> 2024-09-21) (Yahoo error = "1d data not available for startTime=-2208977640 and endTime=1726971606. Only 100 years worth of day granularity data are allowed to be fetched per request.")')
[*********************100%***********************]  831 of 832 completed['BKWB39.SA', 'MRSADEB72.SA', 'CSRNDEB72.SA', 'LUPA14.SA', 'EGIEDEB71.SA', 'RBRACRI77.SA', 'BMFE39.SA', 'PRPT3.SA', 'BHIA12.SA', 'MNZC3.SA', 'BFDA39.SA', 'CEEB6.SA', 'MSRO3.SA', 'AVLL11.SA', 'BFDN39.SA', 'COMR3.SA', 'BCHA39.SA', 'PETRDEB62.SA', 'LLBI3.SA', 'CALI11.SA', 'INEPDCA61.SA', 'AZEV11.SA', 'QUSW3.SA', 'BEQW39.SA', 'DTCY4.SA', 'LTEL3B.SA', 'BBUS39.SA', 'BSCSCRIT4.SA', 'S

In [133]:
historical_data = {}
companies2 = []
data_error = {}
start_time = time.time()
for i, company in enumerate(companies):
    try:
        # Selecionar os dados de company e remover NaNs
        df = data.xs(company, axis=1, level=1).dropna()
        df['value'] = df['Adj Close']

        # get median data from df
        df = get_median_data(df)

        historical_data[company] = df
        companies2.append(company)
        extra_info = [company]
        print_info(i, extra_info, start_time, len(companies))
    except Exception as e:
        data_error[company] = company

0.12% 1+832, 0.087998s per item, Time: 0h 00m 00s + 0h 01m 13s RRRP3.SA
0.60% 5+828, 0.035391s per item, Time: 0h 00m 00s + 0h 00m 29s AERI3.SA
0.72% 6+827, 0.042160s per item, Time: 0h 00m 00s + 0h 00m 34s AESB3.SA
0.96% 8+825, 0.080495s per item, Time: 0h 00m 00s + 0h 01m 06s AFLT3.SA
1.08% 9+824, 0.079111s per item, Time: 0h 00m 00s + 0h 01m 05s AGXY3.SA
1.20% 10+823, 0.127896s per item, Time: 0h 00m 01s + 0h 01m 45s RPAD3.SA
1.32% 11+822, 0.164576s per item, Time: 0h 00m 01s + 0h 02m 15s RPAD5.SA
1.44% 12+821, 0.194028s per item, Time: 0h 00m 02s + 0h 02m 39s RPAD6.SA
1.56% 13+820, 0.192032s per item, Time: 0h 00m 02s + 0h 02m 37s AALR3.SA
1.68% 14+819, 0.183527s per item, Time: 0h 00m 02s + 0h 02m 30s ALLD3.SA
1.80% 15+818, 0.172825s per item, Time: 0h 00m 02s + 0h 02m 21s ALOS3.SA
1.92% 16+817, 0.168208s per item, Time: 0h 00m 02s + 0h 02m 17s ALPK3.SA
2.04% 17+816, 0.159784s per item, Time: 0h 00m 02s + 0h 02m 10s EXCO32.SA
2.16% 18+815, 0.181074s per item, Time: 0h 00m 03s + 0h

In [134]:
# Now find the overall min and max quarter across all tickers
all_quarters = [data['quarter'] for data in historical_data.values()]
max_quarter = pd.concat(all_quarters).max()
previous_quarter = pd.Period(max_quarter, freq='Q') - 1
last_date = previous_quarter.end_time.date()

data2 = yf.download(' '.join(companies2), start=last_date)

[*********************100%***********************]  587 of 587 completed

2 Failed downloads:
['BKYY39.SA', 'YBRA4.SA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-06-30 -> 2024-09-21)')
